In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

#### doji and spinning tops from single candlestick pattern

In [3]:
def identify_dojis(df):
    df['is_doji'] = np.where(
        np.logical_or(
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.15,
                    df['candle_color'] == 'red',
                    ((df['high'] - df['open']) / df['candle_length']).between(.33, .67)
                ]
            ),
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.15,
                    df['candle_color'] == 'green',
                    ((df['high'] - df['close']) / df['candle_length']).between(.33, .67)
                ]
            ),
        ),
        True,
        False
    )
    return df

In [44]:
def identify_spinning_tops(df):
    df['is_spinning_top'] = np.where(
        np.logical_or(
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.4,
                    df['candle_color'] == 'red',
                    ((df['high'] - df['open']) / (df['close'] - df['low'])).between(0.4, 2.5)
                ]
            ),
            np.logical_and.reduce(
                [
                    df['candle_body_ratio'] < 0.4,
                    df['candle_color'] == 'green',
                    ((df['high'] - df['close']) / (df['open'] - df['low'])).between(0.4, 2.5)
                ]
            ),
        ),
        True,
        False
    )
    return df

# Data Loader

In [4]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [66]:
script_name = 'poonawalla'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 370)

In [67]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsep

In [68]:
df

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,low_52w
231,2021-10-13,POONAWALLA,167.70,161.50,167.65,162.15,162.15,164.35,808708,189.2,31.65
232,2021-10-14,POONAWALLA,165.00,161.00,163.40,162.10,162.25,162.15,680816,189.2,31.65
233,2021-10-18,POONAWALLA,165.40,157.60,158.70,160.10,159.65,162.10,4187207,189.2,32.30
234,2021-10-19,POONAWALLA,163.40,155.20,162.00,156.80,156.60,160.10,1060063,189.2,33.10
235,2021-10-20,POONAWALLA,159.15,152.10,155.00,153.30,153.80,156.80,834362,189.2,33.10
...,...,...,...,...,...,...,...,...,...,...,...
1,2022-10-11,POONAWALLA,338.00,325.00,326.95,326.45,326.00,325.70,9309363,343.8,140.75
2,2022-10-12,POONAWALLA,331.30,321.10,329.80,329.30,328.70,326.45,4223879,343.8,140.75
3,2022-10-13,POONAWALLA,330.80,317.85,329.30,321.60,322.05,329.30,2981139,343.8,140.75
4,2022-10-14,POONAWALLA,329.00,318.50,328.00,323.00,324.60,321.60,3131668,343.8,140.75


In [69]:
df['candle_body_length'] = (df['open'] - df['close']).abs()
df['candle_length'] = df['high'] - df['low']
df['candle_body_ratio'] = df['candle_body_length'] / df['candle_length']
df['candle_color'] = np.where(df.close > df.open, 'green', 'red')
df['pct_change'] = (df['close'] - df['prev_close']) * 100 / df['prev_close']

# Multiple candlestick pattern functions

In [70]:
def identify_engulfing(df):
    for i in ['open', 'high', 'close', 'low', 'candle_color', 'candle_body_ratio', 'candle_body_length']:
        df[f'prev_{i}'] = df[i].shift(1)

    df['engulfing'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.15,
            df['prev_open'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            df['prev_close'].between(df[['open', 'close']].min(axis=1), df[['open', 'close']].max(axis=1)),
            (df['prev_candle_body_length'] / df['candle_body_length']) < 0.95
        ]),
        np.where(df["candle_color"] == 'green', "Bullish Engulfing", "Bearish Engulfing"),
        None
    )
    
    return df

def identify_haramis(df):
    df['harami'] = np.where(
        np.logical_and.reduce([
            df['prev_candle_color'] != df['candle_color'],
            df['prev_candle_body_ratio'] >= 0.5,
            (df['candle_body_length'] / df['prev_candle_body_length']) < 0.5,
            df['open'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
            df['close'].between(df[['prev_open', 'prev_close']].min(axis=1), df[['prev_open', 'prev_close']].max(axis=1)),
        ]),
        np.where(df['candle_color'] == 'green', "Bullish Harami", "Bearish Harami"),
        None
    )
    return df

def identify_piercing_or_dark_clouds(df):
    df["partial_engulfing"] = np.where(
        np.logical_and.reduce([
            df["close"].between(df[["prev_open", "prev_close"]].min(axis=1), df[["prev_open", "prev_close"]].max(axis=1)),
            df["prev_candle_body_ratio"] >= 0.5,
            df['candle_body_length'] >= df['prev_candle_body_length'] * 0.5,
            df["candle_color"] != df["prev_candle_color"]
        ]),
        np.where(
            (df["candle_color"] == "green") & (df['close'] >= (df['prev_open'] + df['prev_close']) / 2), 
            "Piercing Pattern",
            np.where(
                (df["candle_color"] == "red") & (df['close'] <= (df['prev_open'] + df['prev_close']) / 2),
                "Dark Cloud Cover",
                None
            )
        ),
        None
    )
    return df

In [71]:
for fn in [identify_engulfing, identify_haramis, identify_piercing_or_dark_clouds]:
    df = fn(df)

In [72]:
test_df = df.copy().set_index('date')

In [73]:
test_df = identify_dojis(test_df)
test_df = identify_spinning_tops(test_df)

In [74]:
test_df['gap_d1'] = test_df['open'] - test_df['prev_close']

In [75]:
test_df['stars'] = np.where(
    np.logical_or(
        np.logical_and.reduce([
            test_df['candle_color'] == 'green',
            test_df['gap_d1'] > 0,
            ((test_df['is_doji'].shift(1) == True) | (test_df['is_spinning_top'].shift(1) == True)),
            test_df['gap_d1'].shift(1) < 0,
            test_df['candle_color'].shift(2) == 'red'
        ]),
        np.logical_and.reduce([
            test_df['candle_color'] == 'red',
            test_df['gap_d1'] < 0,
            ((test_df['is_doji'].shift(1) == True) | (test_df['is_spinning_top'].shift(1) == True)),
            test_df['gap_d1'].shift(1) > 0,
            test_df['candle_color'].shift(2) == 'green'
        ])
    ),
    np.where(test_df['gap_d1'] > 0, 'Bullish morning star', 'Bearish evening star'),
    None
)

In [76]:
test_df[test_df['stars'].notna()]

,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,low_52w,...,prev_candle_color,prev_candle_body_ratio,prev_candle_body_length,engulfing,harami,partial_engulfing,is_doji,is_spinning_top,gap_d1,stars
date,,,,,,,,,,,,,,,,,,,,,
2021-12-21,POONAWALLA,198.70,189.20,191.00,193.55,194.85,188.25,3411927,226.8,35.75,...,red,0.176768,1.75,None,None,None,False,False,2.75,Bullish morning star
2022-01-20,POONAWALLA,288.70,280.55,282.00,283.75,283.80,281.15,3019375,302.9,41.10,...,green,0.230769,3.15,None,None,None,False,False,0.85,Bullish morning star
2022-02-11,POONAWALLA,261.70,253.20,259.60,253.90,254.80,263.55,2150089,302.9,59.15,...,green,0.298182,4.10,None,None,None,False,False,-3.95,Bearish evening star
2022-02-23,POONAWALLA,245.95,238.30,238.45,238.95,238.80,235.95,1547211,302.9,101.55,...,green,0.086758,0.95,None,None,None,False,False,2.50,Bullish morning star
2022-05-19,POONAWALLA,247.30,236.80,243.90,238.90,239.20,253.60,3673019,343.8,127.65,...,red,0.345679,4.20,None,None,None,False,False,-9.70,Bearish evening star
2022-06-28,POONAWALLA,242.20,236.25,239.50,239.00,238.90,240.35,1544925,343.8,140.75,...,green,0.054264,0.35,None,None,None,True,True,-0.85,Bearish evening star
2022-07-25,POONAWALLA,269.70,261.60,268.70,263.05,262.75,269.85,2028612,343.8,140.75,...,red,0.023437,0.15,None,None,None,False,False,-1.15,Bearish evening star


In [26]:
from datetime import datetime, timedelta
import requests
import pandas as pd

# current price url = https://query1.finance.yahoo.com/v8/finance/chart/TEJASNET.NS?region=US&lang=en-US&includePrePost=false&interval=2m&useYfid=true&range=1d&corsDomain=finance.yahoo.com&.tsrc=finance

REQUEST_HEADERS = {
    "user-agent": "Custom"
}

QUERY_PARAMS = {
    "formatted": "true",
    "lang": "en-US",
    "region": "US",
    "includeAdjustedClose": "true",
    "interval": "1d",
    "events": "capitalGain%7Cdiv%7Csplit",
    "useYfid": "true",
    "corsDomain": "finance.yahoo.com",
}

BASE_URL = 'https://query2.finance.yahoo.com/v8/finance/chart/'

column_order = ['date', 'symbol', 'open', 'high', 'low', 'close', 'prev_close', 'volume']


def get_historical_data(script_name:str, from_date:str = None, to_date:str = None, exchg:str='NS'):
    url = f'{BASE_URL}{script_name.upper()}.{exchg.upper()}'
    
    to_timestamp = int((datetime.strptime(to_date, '%Y-%m-%d') if to_date is not None else datetime.now()).timestamp())
    from_timestamp = int((datetime.strptime(from_date, '%Y-%m-%d') if from_date is not None else (datetime.now() - timedelta(days=90))).timestamp())

    query_params = {**QUERY_PARAMS, "period1": from_timestamp, "period2": to_timestamp}

    resp = requests.get(url, headers=REQUEST_HEADERS, params=query_params)

    content = resp.json()['chart']['result'][0]
    data_zip = zip(
        content['timestamp'], 
        content['indicators']['quote'][0]['open'],
        content['indicators']['quote'][0]['high'],
        content['indicators']['quote'][0]['low'],
        content['indicators']['quote'][0]['close'],
        content['indicators']['adjclose'][0]['adjclose'],
        content['indicators']['quote'][0]['volume'],
    )
    df = pd.DataFrame(data_zip, columns=['timestamp', 'open', 'high', 'low', 'close', 'adj_close', 'volume'])
    df['date'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
    df['symbol'] = script_name.upper()
    df['prev_close'] = df['close'].shift(1)

    return df[column_order]

def get_current_price(script_name:str, exchg:str='NS'):
    url = f'{BASE_URL}{script_name.upper()}.{exchg.upper()}'
    query_params = QUERY_PARAMS
    resp = requests.get(url, headers=REQUEST_HEADERS, params=query_params)
    resp_dict = resp.json()['chart']['result'][0]['meta']
    current_price_dict = {k: v for k, v in resp_dict.items() if k in ['regularMarketPrice', 'chartPreviousClose']}
    current_price_dict['change'] = current_price_dict['regularMarketPrice'] - current_price_dict['chartPreviousClose']
    current_price_dict['change_pct'] = current_price_dict['change'] * 100 / current_price_dict['chartPreviousClose']
    
    return current_price_dict

In [27]:
resp_dict = get_current_price('infy')

In [28]:
resp_dict

{'regularMarketPrice': 1496.05,
 'chartPreviousClose': 1503.7,
 'change': -7.650000000000091,
 'change_pct': -0.5087450954312756}